In [1]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1380],
        [-0.2241]], grad_fn=<AddmmBackward0>)

### 参数访问

In [6]:
#我们可以检查第二个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0730,  0.3189, -0.3154, -0.0329,  0.0821, -0.2936, -0.3180, -0.0304]])), ('bias', tensor([-0.1139]))])


In [9]:
"""目标参数"""
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
#查看参数梯度
net[2].weight.grad == None

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1139], requires_grad=True)
tensor([-0.1139])


True

In [13]:
"""一次性访问所有参数"""
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
net.state_dict()['2.bias'].data

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([-0.1139])

In [16]:
"""从嵌套块收集参数"""
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
           nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3450],
        [-0.3451]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
rgnet[0][1][0].bias.data

tensor([ 0.2699, -0.0986, -0.1499, -0.4218, -0.4708,  0.2556,  0.0483, -0.2497])

### 参数初始化 

In [20]:
"""内置初始化"""
#高斯
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0097, -0.0036,  0.0023, -0.0018]), tensor(0.))

In [21]:
#常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [22]:
#以对某些块应用不同的初始化方法
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.2966, -0.4047, -0.3732, -0.3571])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 #绝对值大于等于 5 的元素保留，其余元素置为0
net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[ 0.0000,  0.0000, -6.7281, -0.0000],
        [-7.8151,  0.0000,  7.9472, -6.5222],
        [ 5.7694, -6.1811,  0.0000, -0.0000],
        [ 7.2527, -5.2462, -8.3204,  6.6386],
        [ 0.0000,  5.6638,  0.0000, -0.0000],
        [-7.0575,  9.8408,  6.6045, -0.0000],
        [-0.0000,  0.0000, -5.5978,  9.2765],
        [ 0.0000,  0.0000,  5.5978, -0.0000]], requires_grad=True)

### 参数绑定

In [31]:
"""有时我们希望在多个层间共享参数：我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。"""
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor(100.)
tensor([True, True, True, True, True, True, True, True])
